In [ ]:
import pandas as pd
base = pd.read_csv('census.csv')
base.head()

In [ ]:
base['income'].unique()

In [ ]:
def converte_classe(rotulo):
    if rotulo == ' >50K':
        return 1
    else:
        return 0

In [ ]:
base['income'] = base['income'].apply(converte_classe)

In [ ]:
base.head()

In [ ]:
base['income'].unique()

In [ ]:
X = base.drop('income', axis = 1)

In [ ]:
X.head()

In [ ]:
y = base['income']

In [ ]:
y.head()

In [ ]:
type(y)

In [ ]:
%matplotlib inline
base.age.hist()

In [1]:
import tensorflow as tf

In [ ]:
idade = tf.feature_column.numeric_column('age')

In [ ]:
idade_categorica = [tf.feature_column.bucketized_column(idade, boundaries=[20,30,40,50,60,70,80,90])]

In [ ]:
print(idade_categorica)

In [ ]:
X.columns

In [ ]:
nome_colunas_categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
                           'native-country']

In [ ]:
X['sex'].unique()

In [ ]:
X['workclass'].unique()

In [ ]:
X['native-country'].unique()

In [ ]:
colunas_categoricas = [tf.feature_column.categorical_column_with_vocabulary_list(key = c, vocabulary_list=X[c].unique()) for c in nome_colunas_categoricas]

In [ ]:
print(colunas_categoricas[3])

In [ ]:
nome_colunas_numericas = ['final-weight', 'education-num', 'capital-gain', 'capital-loos', 'hour-per-week']

In [ ]:
colunas_numericas = [ tf.feature_column.numeric_column(key = c) for c in nome_colunas_numericas ]

In [ ]:
print(colunas_numericas[1])

In [ ]:
colunas = idade_categorica + colunas_categoricas + colunas_numericas

In [ ]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3)

In [ ]:
X_treinamento.shape

In [ ]:
X_teste.shape

In [ ]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento, y = y_treinamento,
                                                        batch_size = 32, num_epochs = None, shuffle = True)

In [ ]:
classificador = tf.estimator.LinearClassifier(feature_columns=colunas)

In [ ]:
classificador.train(input_fn=funcao_treinamento, steps = 10000)

In [ ]:
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = X_teste, batch_size = 32, shuffle = False)

In [ ]:
previsoes = classificador.predict(input_fn=funcao_previsao)

In [ ]:
list(previsoes)

In [ ]:
previsoes_final = []
for p in classificador.predict(input_fn=funcao_previsao):
    previsoes_final.append(p['class_ids'])

In [ ]:
previsoes_final

In [ ]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste, previsoes_final)
taxa_acerto